In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import scipy.signal as signal
import scipy.fftpack as fftpack
import timeit, glob
from scipy.signal import butter, lfilter, sosfiltfilt
from matplotlib import rcParams
from matplotlib import pyplot as plt

# Load the big data file downloaded from https://figshare.com/articles/dataset/LFP_data_from_Steinmetz_et_al_2019/9727895
rootPath = "/run/media/h4wke/Elements/LFP/"

Sessions = ['Cori_2016-12-14', 'Cori_2016-12-17', 'Cori_2016-12-18', 'Forssmann_2017-11-01', 'Forssmann_2017-11-02', 'Forssmann_2017-11-04', 'Forssmann_2017-11-05', 'Hench_2017-06-15', 'Hench_2017-06-16', 'Hench_2017-06-17', 'Hench_2017-06-18', 'Lederberg_2017-12-05', 'Lederberg_2017-12-06', 'Lederberg_2017-12-07', 'Lederberg_2017-12-08', 'Lederberg_2017-12-09', 'Lederberg_2017-12-10', 'Lederberg_2017-12-11', 'Moniz_2017-05-15', 'Moniz_2017-05-16', 'Moniz_2017-05-18', 'Muller_2017-01-07', 'Muller_2017-01-08', 'Muller_2017-01-09', 'Radnitz_2017-01-08', 'Radnitz_2017-01-09', 'Radnitz_2017-01-10', 'Radnitz_2017-01-11', 'Radnitz_2017-01-12', 'Richards_2017-10-29', 'Richards_2017-10-30', 'Richards_2017-10-31', 'Richards_2017-11-01', 'Richards_2017-11-02', 'Tatum_2017-12-06', 'Tatum_2017-12-07', 'Tatum_2017-12-08', 'Tatum_2017-12-09', 'Theiler_2017-10-11']

def time_to_sample( time ):
    return round(time * big_lfp_ts[1,0]/big_lfp_ts[1,1])

def samples( trial, intervals, dt ):
    samples = np.array([intervals[trial,0], intervals[trial,1] ])/dt
    samples = samples.astype(int)
    return samples

def filter_data(data, low, high, sf, order=2, axis=-1):
    # Determine Nyquist frequency
    nyq = sf/2

    # Set bands
    low = low/nyq
    high = high/nyq

    # Calculate coefficients
    b, a = butter(order, [low, high], btype='band')

    # Filter signal
    filtered_data = lfilter(b, a, data, axis)

    return filtered_data
export_folder = "/run/media/h4wke/Elements/LFP/250_lfp/"
existing = len(glob.glob(export_folder+"*"))
print(existing)
session = Sessions[existing]
filename = glob.glob(rootPath+session+"_lfp/"+session+"*")[0]
print("Reading: ", filename)
with open(filename, 'r') as f:
    big_lfp = np.fromfile(f, np.int16).reshape((-1, 385)).T

# Get Timestamps and brain region info
dataPath = rootPath + "allData/"

big_lfp_ts = np.load( glob.glob(dataPath+session+"/"+session+"*.imec.lf.timestamps.npy")[0], allow_pickle=True)

rawRow = np.load(glob.glob(dataPath+session+"/channels.rawRow.npy")[0], allow_pickle=True)
intervals = np.load(glob.glob(dataPath+session+"/trials.intervals.npy")[0], allow_pickle=True)
visualTime = np.load(glob.glob(dataPath+session+"/trials.visualStim_times.npy")[0], allow_pickle=True)
df = pd.read_csv(glob.glob(dataPath+session+"/channels.brainLocation.tsv")[0], delimiter="\t")

brainLocation=df.to_numpy()
del df

# Average over all the probes in each region to get a one region to one signal mapping
regions = {}
for i in range(len(brainLocation)):
    row = brainLocation[i]
    if(row[3] == 'root'):
        continue
    if row[3] not in regions:
        regions[row[3]] = { "lfp": np.copy(big_lfp[rawRow[i][0]]), "count": 1, "probes": [i]}
    else:
        regions[row[3]]["lfp"] = regions[row[3]]["lfp"] + big_lfp[rawRow[i][0]]
        regions[row[3]]["count"] += 1
        regions[row[3]]["probes"].append(i)

for r in regions:
    regions[r]["lfp"] = regions[r]["lfp"]/regions[r]["count"]
    regions[r]["lfp"] = signal.detrend( regions[r]["lfp"], type="constant")

fs = 2500
dt = 1/fs
t_trial = 2.5
n_samples = int( t_trial*fs )
export = {'brain_area_lfp': [], 'lfp': np.zeros([len(regions),len(intervals),n_samples])}
i = 0
region_names = list(regions.keys())
for ri in range(len(region_names)):
    region = region_names[ri]
    regions[region]["trials"] = []
    export['brain_area_lfp'].append( region )
    for i in range(len(intervals)):
        start = time_to_sample( visualTime[i][0] )
        trial = regions[region]["lfp"][start:start+n_samples ]
        export['lfp'][ri,i] = trial

export['lfp'] = filter_data(export['lfp'], low=1, high=250, sf=2500, axis=2)
export['lfp'] = signal.decimate(export['lfp'], 5)

print( "Saving " + session + "\nWith data containing " + str( export['brain_area_lfp'] ) + " regions \nShape: " + str( export['lfp'].shape ) )
np.save(export_folder+session+".lfp.processed.npy", export, allow_pickle=True)

39
Reading:  /run/media/h4wke/Elements/LFP/Cori_2016-12-14_lfp/Cori_2016-12-14_M2_g0_t0.imec.lf.bin
